# Predicting Women's March Mania Outcomes

## Introduction

---

Each year, sixty-eight women’s college basketball teams compete in a nationally broadcasted tournament known as March Madness. Alongside the excitement fans share over watching their alma mater’s compete, a different form of competition brews in offices, households, and gambling halls across the country: predicting who will win each matchup and be crowned champion. For this project, we have set out to do just that. 


### Initialize Imports

In [3]:
import pandas as pd
import numpy as np
import os, gc, sys, re, time
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from itertools import combinations
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive')

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO') # not sure what this does, copied from HW 

pd.set_option('display.max_columns', None)

Mounted at /content/gdrive


### Initialize Datasets

In [4]:
tourney_results = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneyCompactResults.csv")
tourney_results_detailed = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneyDetailedResults.csv")

team_names = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WTeamSpellings.csv", encoding="cp1252", 
                         index_col="TeamNameSpelling")
seeds = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneySeeds.csv")
slots = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneySlots.csv")
seasons = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WSeasons.csv")
teams = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WTeams.csv")

# Data Preprocessing

---

Before we begin analysis, we will modify the existing dataset to incorporate and / or calculate various information surrounding a given team. Examples include Field Goals Made and the Score Differential between the winning and losing team of each matchup, average stats of each team during the season, and the Win / Loss Ratio of each team during the season.

## Establishing Team Identifiers and Computing Season Stats

Data will be eventually complied into a new dataframe **season_stats**, representing a summary of statistics surrounding each team for each regular season. *(Note: regular season is played prior to the March Mania tournament)*


### Gathering Regular Season Data 

In [5]:
s_results = pd.read_csv("/content/gdrive/MyDrive/W207/207 Final Project/womens-march-mania-2022/WDataFiles_Stage1/WRegularSeasonDetailedResults.csv")
s_results.shape

(63887, 34)

In [6]:
s_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


## Calculating Field Goal Percentage and Score Differentials

- **Field Goal Percentage:** Field Goals Made / Field Goals Attempted
- **Score Differential:** Winning Team Score - Losing Team Score



In [7]:
s_results['WFGPercent'] = s_results['WFGM'] / s_results['WFGA']
s_results['LFGPercent'] = s_results['LFGM'] / s_results['LFGA']

s_results['WDiff'] = s_results['WScore'] - s_results['LScore']
s_results['LDiff'] = s_results['LScore'] - s_results['WScore']

s_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WFGPercent,LFGPercent,WDiff,LDiff
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19,0.425926,0.370370,14,-14
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27,0.419355,0.396825,5,-5
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15,0.467742,0.327586,12,-12
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14,0.519231,0.243243,5,-5
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14,0.405405,0.337838,4,-4


In [8]:
s_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'WFGPercent', 'LFGPercent', 'WDiff', 'LDiff'],
      dtype='object')

## Removing Win / Loss Identifiers 

The source data is segmented into winning and losing teams, with their stats reflecting their matchup outcome. To make the data more cohesive, we have chosen to blend their stats into a new standard format that does not depend on their win / loss of a matchup.

In [9]:
# Subset Winners + stats
s_winners = s_results[['WTeamID','Season','WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WDiff', 'WFGPercent']]
s_winners.columns = ['TeamID','Season', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Diff', 'FGPercent']
# Subset Losers + stats
s_losers = s_results[['LTeamID','Season','LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR',
       'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LDiff', 'LFGPercent']]
s_losers.columns = ['TeamID','Season', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Diff', 'FGPercent']
# Stack Winner + Loser dataframes
s_all = pd.concat([s_winners, s_losers], ignore_index=True, axis=0)

In [10]:
s_all.head()

,TeamID,Season,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,Diff,FGPercent
0,3103,2010,23,54,5,9,12,19,10,26,14,18,7,0,15,14,0.425926
1,3104,2010,26,62,5,12,16,28,16,31,15,20,5,2,25,5,0.419355
2,3110,2010,29,62,6,15,7,12,14,23,18,13,6,2,17,12,0.467742
3,3111,2010,27,52,4,11,5,9,6,40,14,27,5,10,18,5,0.519231
4,3119,2010,30,74,7,20,7,11,14,33,18,11,5,3,18,4,0.405405


## Calculating Average Stats

Surprisingly the source data does not include stat averages for each team, we have chosen to calculate them ourselves and attach them to each team for each season.

- **Average Stats:** The Mean of Each Statistic for a Given Regular Season


In [11]:
s_stats = s_all.groupby(['TeamID','Season'])[['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Diff', 'FGPercent']].mean().reset_index()
s_stats.head(10)

,TeamID,Season,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,Diff,FGPercent
0,3101,2014,25.086957,59.565217,8.521739,26.652174,11.304348,17.608696,12.217391,23.130435,13.391304,18.304348,8.956522,3.217391,18.521739,0.086957,0.421129
1,3101,2015,21.720000,57.600000,7.280000,26.040000,14.320000,20.440000,15.320000,24.560000,13.040000,18.640000,10.920000,3.400000,17.280000,2.400000,0.374293
2,3101,2016,24.560000,58.320000,7.680000,23.080000,16.400000,23.680000,14.680000,25.120000,14.000000,16.920000,9.560000,4.680000,16.760000,10.000000,0.427180
3,3101,2017,23.962963,57.777778,8.481481,25.555556,14.481481,20.259259,15.666667,24.851852,15.259259,17.259259,7.333333,3.740741,16.888889,6.814815,0.416770
4,3101,2018,22.615385,54.038462,8.461538,25.423077,12.230769,16.384615,12.538462,21.500000,13.307692,17.730769,7.115385,4.076923,18.538462,-2.961538,0.416444
5,3101,2019,25.535714,54.785714,7.892857,23.571429,13.214286,17.714286,11.357143,25.035714,16.464286,18.214286,7.321429,3.392857,18.821429,6.392857,0.468503
6,3101,2020,25.680000,54.840000,9.640000,25.480000,14.920000,19.840000,10.880000,25.480000,15.240000,17.520000,7.240000,3.040000,17.520000,8.200000,0.469022
7,3101,2021,22.052632,51.578947,7.368421,24.052632,12.947368,16.684211,8.526316,24.368421,12.000000,18.631579,6.736842,2.789474,15.157895,-1.526316,0.427403
8,3101,2022,24.052632,56.263158,8.473684,24.157895,13.736842,18.421053,9.473684,20.894737,14.684211,14.736842,6.789474,1.473684,16.736842,2.000000,0.431626
9,3102,2010,19.142857,53.142857,4.571429,16.071429,8.964286,13.214286,12.464286,19.535714,10.250000,17.714286,6.035714,0.642857,14.964286,-19.964286,0.364342


## Cakculating Win Loss Ratio

To round out our newly constructed dataset, we will calculate the Win / Loss Ratio of each team for a given season.

- **Win / Loss Ratio:** Total Wins - Total Losses

In [12]:
w_group = pd.DataFrame(s_winners.groupby(['TeamID','Season'])['TeamID'].count())
w_group.columns= ['Wins']

l_group = pd.DataFrame(s_losers.groupby(['TeamID','Season'])['TeamID'].count())
l_group.columns = ['Losses']

records = pd.concat([w_group,l_group], axis=1,join='outer').reset_index()
records['WLRatio'] = records['Wins'] - records['Losses'] # win loss ratio
records

,TeamID,Season,Wins,Losses,WLRatio
0,3101,2014,11.0,12.0,-1.0
1,3101,2015,13.0,12.0,1.0
2,3101,2016,22.0,3.0,19.0
3,3101,2017,19.0,8.0,11.0
4,3101,2018,12.0,14.0,-2.0
...,...,...,...,...,...
4514,3470,2021,5.0,16.0,-11.0
4515,3470,2022,8.0,10.0,-2.0
4516,3471,2021,6.0,9.0,-3.0
4517,3471,2022,8.0,10.0,-2.0


## Compiling Season Stats

Finally, we merge the Win / Loss Ratio dataframe with our newly constructed dataset and trim down the final dataframe to incorporate the most relevant stats for each team.

In [13]:
season_stats_full = s_stats.merge(records, how='inner', on=['TeamID', 'Season'])
season_stats = season_stats_full[['TeamID','Season', 'Ast', 'FTA', 'PF', 'Diff', 'FGPercent', 'Stl', 'Blk', 'Wins', 'Losses', 'WLRatio']]
season_stats.head()

,TeamID,Season,Ast,FTA,PF,Diff,FGPercent,Stl,Blk,Wins,Losses,WLRatio
0,3101,2014,13.391304,17.608696,18.521739,0.086957,0.421129,8.956522,3.217391,11.0,12.0,-1.0
1,3101,2015,13.040000,20.440000,17.280000,2.400000,0.374293,10.920000,3.400000,13.0,12.0,1.0
2,3101,2016,14.000000,23.680000,16.760000,10.000000,0.427180,9.560000,4.680000,22.0,3.0,19.0
3,3101,2017,15.259259,20.259259,16.888889,6.814815,0.416770,7.333333,3.740741,19.0,8.0,11.0
4,3101,2018,13.307692,16.384615,18.538462,-2.961538,0.416444,7.115385,4.076923,12.0,14.0,-2.0


# Baseline Model 1

---

For our first baseline model, we employ the **tourney_results** dataset which only include 

- Baseline Model 1 uses only season and Team IDs

In [14]:
tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [15]:
tourney_results.shape

(1449, 8)

In [16]:
tourney_results.isna().sum()

Season     0
DayNum     0
WTeamID    0
WScore     0
LTeamID    0
LScore     0
WLoc       0
NumOT      0
dtype: int64

In [17]:
# Team 1 is defined as the Team with the lowest ID number
def lowest_id(row):
  row['Team1'] = min(row['WTeamID'],row['LTeamID'])
  row['Team2'] = max(row['WTeamID'],row['LTeamID'])

  return row

bm1_df = tourney_results.apply(lowest_id, axis=1)
bm1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2
0,1998,137,3104,94,3422,46,H,0,3104,3422
1,1998,137,3112,75,3365,63,H,0,3112,3365
2,1998,137,3163,93,3193,52,H,0,3163,3193
3,1998,137,3198,59,3266,45,H,0,3198,3266
4,1998,137,3203,74,3208,72,A,0,3203,3208


In [18]:
def create_labels_wTeams(row):
  T1 = row['Team1']
  T2 = row['Team2']
  row[str(T1)] = 1
  row[str(T2)] = 1
  if T1 == row['WTeamID']:
    row['Team1_Win'] = 1
  else:
    row['Team1_Win'] = 0
  
  return row

bm1_df = bm1_df.apply(create_labels_wTeams, axis=1)
bm1_df = bm1_df.fillna(0)
bm1_df = bm1_df.drop(['Team1', 'Team2'], axis=1)
bm1_df.head()

,3101,3103,3104,3106,3107,3108,3110,3111,3112,3113,3114,3116,3119,3120,3122,3123,3124,3125,3126,3129,3130,3131,3132,3133,3137,3138,3140,3141,3142,3143,3144,3145,3146,3150,3151,3153,3155,3156,3159,3160,3161,3163,3164,3165,3166,3169,3171,3173,3174,3175,3176,3177,3179,3180,3181,3182,3184,3185,3187,3189,3190,3191,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3205,3207,3208,3209,3210,3211,3212,3214,3216,3217,3218,3219,3221,3222,3224,3225,3226,3228,3229,3231,3233,3234,3235,3238,3239,3241,3242,3243,3245,3246,3249,3250,3251,3252,3253,3254,3256,3257,3258,3261,3263,3264,3265,3266,3268,3269,3270,3272,3273,3274,3275,3276,3277,3278,3279,3280,3281,3283,3285,3286,3291,3292,3293,3294,3298,3299,3301,3304,3307,3308,3311,3313,3314,3315,3318,3319,3320,3321,3322,3323,3324,3325,3326,3328,3329,3330,3331,3332,3333,3335,3336,3337,3338,3340,3341,3343,3345,3346,3347,3349,3350,3352,3353,3355,3357,3359,3360,3361,3362,3364,3365,3366,3369,3370,3371,3372,3373,3374,3376,3377,3378,3380,3382,3383,3384,3385,3386,3388,3389,3390,3391,3392,3393,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3411,3412,3413,3415,3416,3417,3418,3420,3421,3422,3424,3425,3426,3427,3428,3430,3431,3433,3434,3435,3436,3437,3438,3439,3441,3442,3443,3444,3448,3449,3450,3451,3452,3453,3454,3455,3457,3458,3460,3461,3462,3464,DayNum,LScore,LTeamID,NumOT,Season,Team1_Win,WLoc,WScore,WTeamID
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137,46,3422,0,1998,1,H,94,3104
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137,63,3365,0,1998,1,H,75,3112
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [19]:
bm1_df['Team1_Win'].value_counts()

1    769
0    680
Name: Team1_Win, dtype: int64

In [20]:
bm1_df.drop(columns=['DayNum', 'LScore', 'LTeamID', 'NumOT', 'WLoc', 'WScore', 'WTeamID'], inplace=True)

In [21]:
bm1_df.head()

,3101,3103,3104,3106,3107,3108,3110,3111,3112,3113,3114,3116,3119,3120,3122,3123,3124,3125,3126,3129,3130,3131,3132,3133,3137,3138,3140,3141,3142,3143,3144,3145,3146,3150,3151,3153,3155,3156,3159,3160,3161,3163,3164,3165,3166,3169,3171,3173,3174,3175,3176,3177,3179,3180,3181,3182,3184,3185,3187,3189,3190,3191,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3205,3207,3208,3209,3210,3211,3212,3214,3216,3217,3218,3219,3221,3222,3224,3225,3226,3228,3229,3231,3233,3234,3235,3238,3239,3241,3242,3243,3245,3246,3249,3250,3251,3252,3253,3254,3256,3257,3258,3261,3263,3264,3265,3266,3268,3269,3270,3272,3273,3274,3275,3276,3277,3278,3279,3280,3281,3283,3285,3286,3291,3292,3293,3294,3298,3299,3301,3304,3307,3308,3311,3313,3314,3315,3318,3319,3320,3321,3322,3323,3324,3325,3326,3328,3329,3330,3331,3332,3333,3335,3336,3337,3338,3340,3341,3343,3345,3346,3347,3349,3350,3352,3353,3355,3357,3359,3360,3361,3362,3364,3365,3366,3369,3370,3371,3372,3373,3374,3376,3377,3378,3380,3382,3383,3384,3385,3386,3388,3389,3390,3391,3392,3393,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3411,3412,3413,3415,3416,3417,3418,3420,3421,3422,3424,3425,3426,3427,3428,3430,3431,3433,3434,3435,3436,3437,3438,3439,3441,3442,3443,3444,3448,3449,3450,3451,3452,3453,3454,3455,3457,3458,3460,3461,3462,3464,Season,Team1_Win
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1998,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1998,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [22]:
bm1_df_validation = bm1_df.copy()
bm1_df_validation = bm1_df_validation[bm1_df_validation['Season'] >= 2016]
bm1_df = bm1_df[bm1_df['Season'] < 2016]

In [23]:
X_bm1 = bm1_df.drop("Team1_Win", axis=1).values
y_bm1 = bm1_df["Team1_Win"].values

In [24]:
# Train/Test Split
X_train_bm1, X_test_bm1, y_train_bm1, y_test_bm1 = train_test_split(X_bm1, y_bm1, test_size=0.3, random_state=42)

In [25]:
bm1_model = LogisticRegression(max_iter=500) # initialize logistic regression model

In [26]:
bm1_model.fit(X_train_bm1, y_train_bm1) # fit model

LogisticRegression(max_iter=500)

In [27]:
y_pred_bm1 = bm1_model.predict(X_test_bm1)

## Baseline Model 1 Metrics

In [28]:
bm1_rmse = mean_squared_error(y_test_bm1, y_pred_bm1, squared=False)
print(f"RMSE: {bm1_rmse:.3f}")
print(classification_report(y_test_bm1, y_pred_bm1))

RMSE: 0.665
              precision    recall  f1-score   support

           0       0.51      0.48      0.49       153
           1       0.59      0.62      0.61       188

    accuracy                           0.56       341
   macro avg       0.55      0.55      0.55       341
weighted avg       0.55      0.56      0.56       341



# Baseline Model 2

- Baseline Model 2 uses only season and win/loss ratio with the assumption:
  higher W/L ratio = winner
- Does NOT include Team IDs

In [29]:
tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [30]:
bm2_df = tourney_results.apply(lowest_id, axis=1)
bm2_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2
0,1998,137,3104,94,3422,46,H,0,3104,3422
1,1998,137,3112,75,3365,63,H,0,3112,3365
2,1998,137,3163,93,3193,52,H,0,3163,3193
3,1998,137,3198,59,3266,45,H,0,3198,3266
4,1998,137,3203,74,3208,72,A,0,3203,3208


Create 'NoId' dataframe for baseline w/o team IDs

In [31]:
def create_labels_noTeams(row):
  T1 = row['Team1']
  T2 = row['Team2']
  if T1 == row['WTeamID']:
    row['Team1_Win'] = 1
  else:
    row['Team1_Win'] = 0
  
  return row

bm2_df = bm2_df.apply(create_labels_noTeams, axis=1)
bm2_df.head(30)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win
0,1998,137,3104,94,3422,46,H,0,3104,3422,1
1,1998,137,3112,75,3365,63,H,0,3112,3365,1
2,1998,137,3163,93,3193,52,H,0,3163,3193,1
3,1998,137,3198,59,3266,45,H,0,3198,3266,1
4,1998,137,3203,74,3208,72,A,0,3203,3208,1
5,1998,137,3234,77,3269,59,H,0,3234,3269,1
6,1998,137,3242,72,3408,68,H,0,3242,3408,1
7,1998,137,3301,89,3263,64,H,0,3263,3301,0
8,1998,137,3304,76,3307,59,N,0,3304,3307,1
9,1998,137,3314,91,3224,71,H,0,3224,3314,0


Join `bm2_df` with `season_stats`

In [32]:
bm2_df = bm2_df.merge(season_stats, left_on=['Team1','Season'], right_on=['TeamID', 'Season'], how='inner')
bm2_df = bm2_df.drop('TeamID', axis=1)
bm2_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,Ast,FTA,PF,Diff,FGPercent,Stl,Blk,Wins,Losses,WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
3,2010,147,3124,51,3181,48,N,0,3124,3181,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
4,2010,153,3163,70,3124,50,N,0,3124,3163,0,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0


In [33]:
bm2_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'Ast', 'FTA', 'PF', 'Diff',
       'FGPercent', 'Stl', 'Blk', 'Wins', 'Losses', 'WLRatio'],
      dtype='object')

In [34]:
bm2_df.columns = ['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1Ast', 'T1FTA', 'T1PF', 'T1Diff',
       'T1FGPercent', 'T1Stl', 'T1Blk', 'T1Wins', 'T1Losses', 'T1WLRatio']

bm2_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1Ast,T1FTA,T1PF,T1Diff,T1FGPercent,T1Stl,T1Blk,T1Wins,T1Losses,T1WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
3,2010,147,3124,51,3181,48,N,0,3124,3181,1,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0
4,2010,153,3163,70,3124,50,N,0,3124,3163,0,14.6875,24.0,14.125,15.25,0.462367,6.96875,7.25,23.0,9.0,14.0


In [35]:
bm2_df = bm2_df.merge(season_stats, left_on =['Team2','Season'], right_on=['TeamID', 'Season'], how='inner')
bm2_df = bm2_df.drop('TeamID', axis=1)
bm2_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1Ast,T1FTA,T1PF,T1Diff,T1FGPercent,T1Stl,T1Blk,T1Wins,T1Losses,T1WLRatio,Ast,FTA,PF,Diff,FGPercent,Stl,Blk,Wins,Losses,WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,14.636364,18.242424,16.393939,12.878788,0.420898,10.393939,2.606061,27.0,6.0,21.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.866667,19.533333,18.033333,9.666667,0.397236,13.366667,2.600000,24.0,6.0,18.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0
3,2010,140,3397,92,3173,64,H,0,3173,3397,0,14.461538,19.346154,16.576923,11.269231,0.420849,7.730769,4.076923,21.0,5.0,16.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0
4,2010,138,3397,75,3122,42,H,0,3122,3397,0,10.156250,23.625000,17.406250,-5.593750,0.388526,7.250000,2.906250,15.0,17.0,-2.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0


In [36]:
bm2_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1Ast', 'T1FTA', 'T1PF',
       'T1Diff', 'T1FGPercent', 'T1Stl', 'T1Blk', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'Ast', 'FTA', 'PF', 'Diff', 'FGPercent', 'Stl', 'Blk',
       'Wins', 'Losses', 'WLRatio'],
      dtype='object')

In [37]:
bm2_df.columns = ['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1Ast', 'T1FTA', 'T1PF',
       'T1Diff', 'T1FGPercent', 'T1Stl', 'T1Blk', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'T2Ast', 'T2FTA', 'T2PF', 'T2Diff', 'T2FGPercent', 'T2Stl', 'T2Blk',
       'T2Wins', 'T2Losses', 'T2WLRatio']
bm2_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1Ast,T1FTA,T1PF,T1Diff,T1FGPercent,T1Stl,T1Blk,T1Wins,T1Losses,T1WLRatio,T2Ast,T2FTA,T2PF,T2Diff,T2FGPercent,T2Stl,T2Blk,T2Wins,T2Losses,T2WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,14.636364,18.242424,16.393939,12.878788,0.420898,10.393939,2.606061,27.0,6.0,21.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.866667,19.533333,18.033333,9.666667,0.397236,13.366667,2.600000,24.0,6.0,18.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0
3,2010,140,3397,92,3173,64,H,0,3173,3397,0,14.461538,19.346154,16.576923,11.269231,0.420849,7.730769,4.076923,21.0,5.0,16.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0
4,2010,138,3397,75,3122,42,H,0,3122,3397,0,10.156250,23.625000,17.406250,-5.593750,0.388526,7.250000,2.906250,15.0,17.0,-2.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0


Remove TeamIDs

In [38]:
bm2_df.columns = bm2_df.columns.str.replace('TeamID_Win','Team1_Win')
# remove columns containing TeamID info
bm2_df = bm2_df[['Season', 'DayNum', 'WLoc',
       'Team1_Win', 'T1Ast', 'T1FTA', 'T1PF',
       'T1Diff', 'T1FGPercent', 'T1Stl', 'T1Blk', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'T2Ast', 'T2FTA', 'T2PF', 'T2Diff', 'T2FGPercent', 'T2Stl', 'T2Blk',
       'T2Wins', 'T2Losses', 'T2WLRatio']]

bm2_df.head()

,Season,DayNum,WLoc,Team1_Win,T1Ast,T1FTA,T1PF,T1Diff,T1FGPercent,T1Stl,T1Blk,T1Wins,T1Losses,T1WLRatio,T2Ast,T2FTA,T2PF,T2Diff,T2FGPercent,T2Stl,T2Blk,T2Wins,T2Losses,T2WLRatio
0,2010,138,N,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,14.636364,18.242424,16.393939,12.878788,0.420898,10.393939,2.606061,27.0,6.0,21.0
1,2010,140,N,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.866667,19.533333,18.033333,9.666667,0.397236,13.366667,2.600000,24.0,6.0,18.0
2,2010,145,A,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0
3,2010,140,H,0,14.461538,19.346154,16.576923,11.269231,0.420849,7.730769,4.076923,21.0,5.0,16.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0
4,2010,138,H,0,10.156250,23.625000,17.406250,-5.593750,0.388526,7.250000,2.906250,15.0,17.0,-2.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0


Filter out missing data *(Note: No regular season provided prior to 2010)*

In [39]:
print(f"Starting rows: {bm2_df.shape[0]}")
bm2_df = bm2_df.dropna()
print(bm2_df.isna().sum())
print(f"Num rows after dropna: {bm2_df.shape[0]}")

Starting rows: 693
Season         0
DayNum         0
WLoc           0
Team1_Win      0
T1Ast          0
T1FTA          0
T1PF           0
T1Diff         0
T1FGPercent    0
T1Stl          0
T1Blk          0
T1Wins         0
T1Losses       0
T1WLRatio      0
T2Ast          0
T2FTA          0
T2PF           0
T2Diff         0
T2FGPercent    0
T2Stl          0
T2Blk          0
T2Wins         0
T2Losses       0
T2WLRatio      0
dtype: int64
Num rows after dropna: 652


Dummy Code 'WLoc' (location of winning team: H-home, A-away, N-neutral)

In [40]:
def loc_convert(row):
  if row['Team1_Win'] == 1:
    if row['WLoc'] == 'H':
      row['Team1_Home'] = 1
      row['Team1_Neutral'] = 0
    elif row['WLoc'] == 'A':
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 0
    else:
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 1
  else:
    if row['WLoc'] == 'A':
      row['Team1_Home'] = 1
      row['Team1_Neutral'] = 0
    elif row['WLoc'] == 'H':
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 0
    else:
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 1

  return row

bm2_df = bm2_df.apply(loc_convert, axis=1)
bm2_df = bm2_df.drop('WLoc', axis=1)
bm2_df.head()

,Season,DayNum,Team1_Win,T1Ast,T1FTA,T1PF,T1Diff,T1FGPercent,T1Stl,T1Blk,T1Wins,T1Losses,T1WLRatio,T2Ast,T2FTA,T2PF,T2Diff,T2FGPercent,T2Stl,T2Blk,T2Wins,T2Losses,T2WLRatio,Team1_Home,Team1_Neutral
0,2010,138,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,14.636364,18.242424,16.393939,12.878788,0.420898,10.393939,2.606061,27.0,6.0,21.0,0,1
1,2010,140,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.866667,19.533333,18.033333,9.666667,0.397236,13.366667,2.600000,24.0,6.0,18.0,0,1
2,2010,145,1,14.687500,24.000000,14.125000,15.250000,0.462367,6.968750,7.250000,23.0,9.0,14.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0,0,0
3,2010,140,0,14.461538,19.346154,16.576923,11.269231,0.420849,7.730769,4.076923,21.0,5.0,16.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0,0,0
4,2010,138,0,10.156250,23.625000,17.406250,-5.593750,0.388526,7.250000,2.906250,15.0,17.0,-2.0,15.593750,17.781250,14.906250,17.406250,0.461583,6.812500,6.500000,30.0,2.0,28.0,0,0


`bm2_df` includes:
  - Season and DayNum 
  - Team1 regular season stats, Team2 regular season stats, Team1 Home or Neutral, Team1_Win 
  - 2010-2021 only (no regular season data prior to 2010) 

Create Train/Test split for 'No ID' data:

In [41]:
bm2_df_validation = bm2_df[bm2_df['Season'] >= 2016]
bm2_df = bm2_df[bm2_df['Season'] < 2016]

In [42]:
# split into X and Y
X_bm2 = bm2_df.drop('Team1_Win',axis=1)
Y_bm2 = bm2_df['Team1_Win']
print(X_bm2.shape)
print(Y_bm2.shape)
# split into test/train
X_train_bm2, X_test_bm2, Y_train_bm2, Y_test_bm2 = train_test_split(X_bm2, Y_bm2, test_size=0.3, random_state=42)

(353, 24)
(353,)


In [43]:
X_test_bm2

,Season,DayNum,T1Ast,T1FTA,T1PF,T1Diff,T1FGPercent,T1Stl,T1Blk,T1Wins,T1Losses,T1WLRatio,T2Ast,T2FTA,T2PF,T2Diff,T2FGPercent,T2Stl,T2Blk,T2Wins,T2Losses,T2WLRatio,Team1_Home,Team1_Neutral
231,2013,141,21.843750,20.343750,11.906250,26.875000,0.516604,9.343750,6.750000,31.0,1.0,30.0,15.300000,21.733333,14.300000,10.233333,0.455868,9.933333,3.633333,21.0,9.0,12.0,1,0
48,2010,139,11.766667,22.866667,18.066667,7.166667,0.400294,10.966667,3.900000,21.0,9.0,12.0,18.833333,18.500000,17.433333,15.833333,0.466499,12.266667,2.033333,26.0,4.0,22.0,0,1
223,2013,138,13.562500,17.687500,15.406250,9.625000,0.424594,10.875000,4.531250,21.0,11.0,10.0,14.218750,19.562500,16.406250,6.500000,0.429674,5.968750,3.750000,22.0,10.0,12.0,0,1
205,2013,141,15.937500,17.062500,16.343750,5.437500,0.424687,7.500000,5.656250,20.0,12.0,8.0,19.218750,22.500000,13.968750,23.093750,0.457097,10.937500,3.906250,31.0,1.0,30.0,1,0
62,2010,141,17.937500,21.812500,16.968750,14.968750,0.452205,12.718750,3.750000,27.0,5.0,22.0,12.032258,18.419355,15.225806,11.000000,0.408696,8.709677,3.516129,25.0,6.0,19.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,2012,140,16.718750,18.375000,13.875000,22.218750,0.454298,6.781250,4.562500,31.0,1.0,30.0,13.687500,22.125000,18.187500,12.625000,0.398158,9.312500,4.531250,23.0,9.0,14.0,0,1
232,2013,139,16.931034,17.206897,15.586207,18.206897,0.439383,9.482759,3.379310,27.0,2.0,25.0,11.866667,18.933333,14.800000,5.100000,0.396744,7.900000,5.833333,18.0,12.0,6.0,0,0
376,2015,139,14.193548,20.096774,18.032258,10.258065,0.435401,8.838710,5.419355,24.0,7.0,17.0,18.517241,14.827586,14.965517,15.310345,0.453943,5.448276,5.793103,25.0,4.0,21.0,0,1
280,2014,138,15.281250,16.843750,14.843750,14.125000,0.452348,8.343750,4.812500,29.0,3.0,26.0,14.322581,19.096774,17.096774,9.612903,0.398375,10.096774,4.387097,22.0,9.0,13.0,0,1


In [44]:
# BASELINE: predict winner as the team with higher W/L Ratio
def bm2_model(row):
  WL1 = row['T1WLRatio']
  WL2 = row['T2WLRatio']
  predictions = []
  if WL1 > WL2:
    predictions.append(1)
  else:
    predictions.append(0)
  
  return predictions

y_pred_bm2 = X_test_bm2.apply(bm2_model, axis=1)

## Baseline Model 2 Metrics

In [45]:
bm2_rmse = mean_squared_error(list(Y_test_bm2), list(y_pred_bm2), squared=False)
print(f"RMSE: {bm2_rmse:.3f}")
print(classification_report(list(Y_test_bm2), list(y_pred_bm2)))

RMSE: 0.637
              precision    recall  f1-score   support

           0       0.57      0.61      0.59        51
           1       0.62      0.58      0.60        55

    accuracy                           0.59       106
   macro avg       0.59      0.59      0.59       106
weighted avg       0.60      0.59      0.59       106



# Model 1
- Makes use of all regular season statistics 

In [46]:
tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [47]:
# Use lowest_id to set lowest ID team to Team1
m1_df = tourney_results.copy()
m1_df = m1_df.apply(lowest_id, axis=1)
m1_df = m1_df.apply(create_labels_noTeams, axis=1)
m1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win
0,1998,137,3104,94,3422,46,H,0,3104,3422,1
1,1998,137,3112,75,3365,63,H,0,3112,3365,1
2,1998,137,3163,93,3193,52,H,0,3163,3193,1
3,1998,137,3198,59,3266,45,H,0,3198,3266,1
4,1998,137,3203,74,3208,72,A,0,3203,3208,1


In [48]:
season_stats_full.head()

,TeamID,Season,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,Diff,FGPercent,Wins,Losses,WLRatio
0,3101,2014,25.086957,59.565217,8.521739,26.652174,11.304348,17.608696,12.217391,23.130435,13.391304,18.304348,8.956522,3.217391,18.521739,0.086957,0.421129,11.0,12.0,-1.0
1,3101,2015,21.720000,57.600000,7.280000,26.040000,14.320000,20.440000,15.320000,24.560000,13.040000,18.640000,10.920000,3.400000,17.280000,2.400000,0.374293,13.0,12.0,1.0
2,3101,2016,24.560000,58.320000,7.680000,23.080000,16.400000,23.680000,14.680000,25.120000,14.000000,16.920000,9.560000,4.680000,16.760000,10.000000,0.427180,22.0,3.0,19.0
3,3101,2017,23.962963,57.777778,8.481481,25.555556,14.481481,20.259259,15.666667,24.851852,15.259259,17.259259,7.333333,3.740741,16.888889,6.814815,0.416770,19.0,8.0,11.0
4,3101,2018,22.615385,54.038462,8.461538,25.423077,12.230769,16.384615,12.538462,21.500000,13.307692,17.730769,7.115385,4.076923,18.538462,-2.961538,0.416444,12.0,14.0,-2.0


In [49]:
season_stats_full.columns

Index(['TeamID', 'Season', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Diff', 'FGPercent', 'Wins',
       'Losses', 'WLRatio'],
      dtype='object')

In [50]:
m1_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win'],
      dtype='object')

Merge with regular season stats

In [51]:
s_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WFGPercent,LFGPercent,WDiff,LDiff
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19,0.425926,0.370370,14,-14
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27,0.419355,0.396825,5,-5
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15,0.467742,0.327586,12,-12
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14,0.519231,0.243243,5,-5
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14,0.405405,0.337838,4,-4


In [52]:
m1_df = m1_df.merge(season_stats_full, left_on =['Team1','Season'], right_on=['TeamID', 'Season'], how='inner')
m1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,Diff,FGPercent,Wins,Losses,WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,3124,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,3124,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,3124,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
3,2010,147,3124,51,3181,48,N,0,3124,3181,1,3124,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
4,2010,153,3163,70,3124,50,N,0,3124,3163,0,3124,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0


In [53]:
m1_df = m1_df.drop('TeamID', axis=1)
m1_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'FGM', 'FGA', 'FGM3', 'FGA3',
       'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Diff',
       'FGPercent', 'Wins', 'Losses', 'WLRatio'],
      dtype='object')

In [54]:
m1_df.columns = ['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1FGM', 'T1FGA', 'T1FGM3', 'T1FGA3',
       'T1FTM', 'T1FTA', 'T1OR', 'T1DR', 'T1Ast', 'T1TO', 'T1Stl', 'T1Blk', 'T1PF', 'T1Diff',
       'T1FGPercent', 'T1Wins', 'T1Losses', 'T1WLRatio']
m1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1FGM,T1FGA,T1FGM3,T1FGA3,T1FTM,T1FTA,T1OR,T1DR,T1Ast,T1TO,T1Stl,T1Blk,T1PF,T1Diff,T1FGPercent,T1Wins,T1Losses,T1WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
3,2010,147,3124,51,3181,48,N,0,3124,3181,1,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0
4,2010,153,3163,70,3124,50,N,0,3124,3163,0,26.09375,56.34375,2.46875,8.65625,17.4375,24.0,12.1875,31.0625,14.6875,16.875,6.96875,7.25,14.125,15.25,0.462367,23.0,9.0,14.0


In [55]:
m1_df = m1_df.merge(season_stats_full, left_on =['Team2','Season'], right_on=['TeamID', 'Season'], how='inner')
m1_df = m1_df.drop('TeamID', axis=1)
m1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1FGM,T1FGA,T1FGM3,T1FGA3,T1FTM,T1FTA,T1OR,T1DR,T1Ast,T1TO,T1Stl,T1Blk,T1PF,T1Diff,T1FGPercent,T1Wins,T1Losses,T1WLRatio,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,Diff,FGPercent,Wins,Losses,WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,25.848485,61.575758,8.818182,24.848485,12.939394,18.242424,13.878788,24.575758,14.636364,15.636364,10.393939,2.606061,16.393939,12.878788,0.420898,27.0,6.0,21.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,23.833333,60.266667,6.333333,20.066667,14.033333,19.533333,15.766667,19.466667,15.866667,16.300000,13.366667,2.600000,18.033333,9.666667,0.397236,24.0,6.0,18.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0
3,2010,140,3397,92,3173,64,H,0,3173,3397,0,25.884615,61.538462,5.461538,17.461538,13.538462,19.346154,14.730769,28.076923,14.461538,16.961538,7.730769,4.076923,16.576923,11.269231,0.420849,21.0,5.0,16.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0
4,2010,138,3397,75,3122,42,H,0,3122,3397,0,22.906250,59.156250,4.156250,15.406250,16.218750,23.625000,14.531250,26.250000,10.156250,18.281250,7.250000,2.906250,17.406250,-5.593750,0.388526,15.0,17.0,-2.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0


In [56]:
m1_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1FGM', 'T1FGA', 'T1FGM3',
       'T1FGA3', 'T1FTM', 'T1FTA', 'T1OR', 'T1DR', 'T1Ast', 'T1TO', 'T1Stl',
       'T1Blk', 'T1PF', 'T1Diff', 'T1FGPercent', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Diff', 'FGPercent', 'Wins', 'Losses',
       'WLRatio'],
      dtype='object')

In [57]:
m1_df.columns = ['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1FGM', 'T1FGA', 'T1FGM3',
       'T1FGA3', 'T1FTM', 'T1FTA', 'T1OR', 'T1DR', 'T1Ast', 'T1TO', 'T1Stl',
       'T1Blk', 'T1PF', 'T1Diff', 'T1FGPercent', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'T2FGM', 'T2FGA', 'T2FGM3', 'T2FGA3', 'T2FTM', 'T2FTA', 'T2OR', 'T2DR',
       'T2Ast', 'T2TO', 'T2Stl', 'T2Blk', 'T2PF', 'T2Diff', 'T2FGPercent', 'T2Wins', 'T2Losses',
       'T2WLRatio']

m1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1FGM,T1FGA,T1FGM3,T1FGA3,T1FTM,T1FTA,T1OR,T1DR,T1Ast,T1TO,T1Stl,T1Blk,T1PF,T1Diff,T1FGPercent,T1Wins,T1Losses,T1WLRatio,T2FGM,T2FGA,T2FGM3,T2FGA3,T2FTM,T2FTA,T2OR,T2DR,T2Ast,T2TO,T2Stl,T2Blk,T2PF,T2Diff,T2FGPercent,T2Wins,T2Losses,T2WLRatio
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,25.848485,61.575758,8.818182,24.848485,12.939394,18.242424,13.878788,24.575758,14.636364,15.636364,10.393939,2.606061,16.393939,12.878788,0.420898,27.0,6.0,21.0
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,23.833333,60.266667,6.333333,20.066667,14.033333,19.533333,15.766667,19.466667,15.866667,16.300000,13.366667,2.600000,18.033333,9.666667,0.397236,24.0,6.0,18.0
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0
3,2010,140,3397,92,3173,64,H,0,3173,3397,0,25.884615,61.538462,5.461538,17.461538,13.538462,19.346154,14.730769,28.076923,14.461538,16.961538,7.730769,4.076923,16.576923,11.269231,0.420849,21.0,5.0,16.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0
4,2010,138,3397,75,3122,42,H,0,3122,3397,0,22.906250,59.156250,4.156250,15.406250,16.218750,23.625000,14.531250,26.250000,10.156250,18.281250,7.250000,2.906250,17.406250,-5.593750,0.388526,15.0,17.0,-2.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0


In [58]:
def add_seeds(row):
  season = row['Season']
  T1 = row['Team1']
  T2 = row['Team2']
  row['T1Seed'] = seeds.query("Season=={0} & TeamID=={1}".format(season, T1))['Seed'].values[0][1:].lstrip('0')
  row['T2Seed'] = seeds.query("Season=={0} & TeamID=={1}".format(season, T2))['Seed'].values[0][1:].lstrip('0')

  return row

m1_df = m1_df.apply(add_seeds, axis=1)
m1_df[['T1Seed', 'T2Seed']] = m1_df[['T1Seed', 'T2Seed']].astype('int')
m1_df['Season'] = m1_df['Season'].astype('int')
m1_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Win,T1FGM,T1FGA,T1FGM3,T1FGA3,T1FTM,T1FTA,T1OR,T1DR,T1Ast,T1TO,T1Stl,T1Blk,T1PF,T1Diff,T1FGPercent,T1Wins,T1Losses,T1WLRatio,T2FGM,T2FGA,T2FGM3,T2FGA3,T2FTM,T2FTA,T2OR,T2DR,T2Ast,T2TO,T2Stl,T2Blk,T2PF,T2Diff,T2FGPercent,T2Wins,T2Losses,T2WLRatio,T1Seed,T2Seed
0,2010,138,3124,69,3201,55,N,0,3124,3201,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,25.848485,61.575758,8.818182,24.848485,12.939394,18.242424,13.878788,24.575758,14.636364,15.636364,10.393939,2.606061,16.393939,12.878788,0.420898,27.0,6.0,21.0,4,13
1,2010,140,3124,49,3207,33,N,0,3124,3207,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,23.833333,60.266667,6.333333,20.066667,14.033333,19.533333,15.766667,19.466667,15.866667,16.300000,13.366667,2.600000,18.033333,9.666667,0.397236,24.0,6.0,18.0,4,5
2,2010,145,3124,77,3397,62,A,0,3124,3397,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,4,1
3,2010,140,3397,92,3173,64,H,0,3173,3397,0,25.884615,61.538462,5.461538,17.461538,13.538462,19.346154,14.730769,28.076923,14.461538,16.961538,7.730769,4.076923,16.576923,11.269231,0.420849,21.0,5.0,16.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,8,1
4,2010,138,3397,75,3122,42,H,0,3122,3397,0,22.906250,59.156250,4.156250,15.406250,16.218750,23.625000,14.531250,26.250000,10.156250,18.281250,7.250000,2.906250,17.406250,-5.593750,0.388526,15.0,17.0,-2.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,16,1


Drop TeamID columns:

In [59]:
m1_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'Team1', 'Team2', 'Team1_Win', 'T1FGM', 'T1FGA', 'T1FGM3',
       'T1FGA3', 'T1FTM', 'T1FTA', 'T1OR', 'T1DR', 'T1Ast', 'T1TO', 'T1Stl',
       'T1Blk', 'T1PF', 'T1Diff', 'T1FGPercent', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'T2FGM', 'T2FGA', 'T2FGM3', 'T2FGA3', 'T2FTM', 'T2FTA',
       'T2OR', 'T2DR', 'T2Ast', 'T2TO', 'T2Stl', 'T2Blk', 'T2PF', 'T2Diff',
       'T2FGPercent', 'T2Wins', 'T2Losses', 'T2WLRatio', 'T1Seed', 'T2Seed'],
      dtype='object')

In [60]:
m1_df = m1_df[['Season', 'DayNum', 'WLoc',
       'Team1_Win', 'T1FGM', 'T1FGA', 'T1FGM3',
       'T1FGA3', 'T1FTM', 'T1FTA', 'T1OR', 'T1DR', 'T1Ast', 'T1TO', 'T1Stl',
       'T1Blk', 'T1PF', 'T1Diff', 'T1FGPercent', 'T1Wins', 'T1Losses',
       'T1WLRatio', 'T2FGM', 'T2FGA', 'T2FGM3', 'T2FGA3', 'T2FTM', 'T2FTA',
       'T2OR', 'T2DR', 'T2Ast', 'T2TO', 'T2Stl', 'T2Blk', 'T2PF', 'T2Diff',
       'T2FGPercent', 'T2Wins', 'T2Losses', 'T2WLRatio', 'T1Seed', 'T2Seed']]
m1_df.head()

,Season,DayNum,WLoc,Team1_Win,T1FGM,T1FGA,T1FGM3,T1FGA3,T1FTM,T1FTA,T1OR,T1DR,T1Ast,T1TO,T1Stl,T1Blk,T1PF,T1Diff,T1FGPercent,T1Wins,T1Losses,T1WLRatio,T2FGM,T2FGA,T2FGM3,T2FGA3,T2FTM,T2FTA,T2OR,T2DR,T2Ast,T2TO,T2Stl,T2Blk,T2PF,T2Diff,T2FGPercent,T2Wins,T2Losses,T2WLRatio,T1Seed,T2Seed
0,2010,138,N,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,25.848485,61.575758,8.818182,24.848485,12.939394,18.242424,13.878788,24.575758,14.636364,15.636364,10.393939,2.606061,16.393939,12.878788,0.420898,27.0,6.0,21.0,4,13
1,2010,140,N,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,23.833333,60.266667,6.333333,20.066667,14.033333,19.533333,15.766667,19.466667,15.866667,16.300000,13.366667,2.600000,18.033333,9.666667,0.397236,24.0,6.0,18.0,4,5
2,2010,145,A,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,4,1
3,2010,140,H,0,25.884615,61.538462,5.461538,17.461538,13.538462,19.346154,14.730769,28.076923,14.461538,16.961538,7.730769,4.076923,16.576923,11.269231,0.420849,21.0,5.0,16.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,8,1
4,2010,138,H,0,22.906250,59.156250,4.156250,15.406250,16.218750,23.625000,14.531250,26.250000,10.156250,18.281250,7.250000,2.906250,17.406250,-5.593750,0.388526,15.0,17.0,-2.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,16,1


In [61]:
def loc_convert(row):
  if row['Team1_Win'] == 1:
    if row['WLoc'] == 'H':
      row['Team1_Home'] = 1
      row['Team1_Neutral'] = 0
    elif row['WLoc'] == 'A':
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 0
    else:
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 1
  else:
    if row['WLoc'] == 'A':
      row['Team1_Home'] = 1
      row['Team1_Neutral'] = 0
    elif row['WLoc'] == 'H':
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 0
    else:
      row['Team1_Home'] = 0
      row['Team1_Neutral'] = 1

  return row

m1_df = m1_df.apply(loc_convert, axis=1)
m1_df = m1_df.drop('WLoc', axis=1)
m1_df.head()

,Season,DayNum,Team1_Win,T1FGM,T1FGA,T1FGM3,T1FGA3,T1FTM,T1FTA,T1OR,T1DR,T1Ast,T1TO,T1Stl,T1Blk,T1PF,T1Diff,T1FGPercent,T1Wins,T1Losses,T1WLRatio,T2FGM,T2FGA,T2FGM3,T2FGA3,T2FTM,T2FTA,T2OR,T2DR,T2Ast,T2TO,T2Stl,T2Blk,T2PF,T2Diff,T2FGPercent,T2Wins,T2Losses,T2WLRatio,T1Seed,T2Seed,Team1_Home,Team1_Neutral
0,2010,138,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,25.848485,61.575758,8.818182,24.848485,12.939394,18.242424,13.878788,24.575758,14.636364,15.636364,10.393939,2.606061,16.393939,12.878788,0.420898,27.0,6.0,21.0,4,13,0,1
1,2010,140,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,23.833333,60.266667,6.333333,20.066667,14.033333,19.533333,15.766667,19.466667,15.866667,16.300000,13.366667,2.600000,18.033333,9.666667,0.397236,24.0,6.0,18.0,4,5,0,1
2,2010,145,1,26.093750,56.343750,2.468750,8.656250,17.437500,24.000000,12.187500,31.062500,14.687500,16.875000,6.968750,7.250000,14.125000,15.250000,0.462367,23.0,9.0,14.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,4,1,0,0
3,2010,140,0,25.884615,61.538462,5.461538,17.461538,13.538462,19.346154,14.730769,28.076923,14.461538,16.961538,7.730769,4.076923,16.576923,11.269231,0.420849,21.0,5.0,16.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,8,1,0,0
4,2010,138,0,22.906250,59.156250,4.156250,15.406250,16.218750,23.625000,14.531250,26.250000,10.156250,18.281250,7.250000,2.906250,17.406250,-5.593750,0.388526,15.0,17.0,-2.0,28.343750,61.312500,5.343750,14.125000,12.062500,17.781250,15.312500,28.000000,15.593750,14.906250,6.812500,6.500000,14.906250,17.406250,0.461583,30.0,2.0,28.0,16,1,0,0


In [62]:
m1_df = m1_df.dropna()

In [63]:
m1_df_validation = m1_df[m1_df['Season'] >= 2016]
m1_df = m1_df[m1_df['Season'] < 2016]

In [64]:
X_m1 = m1_df.drop(["Team1_Win"], axis=1).values
y_m1 = m1_df["Team1_Win"].values

In [65]:
X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split(X_m1, y_m1, test_size=0.3, random_state=42)

In [66]:
m1_model = LogisticRegression(max_iter=500)

In [67]:
m1_model.fit(X_train_m1, y_train_m1)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [68]:
y_pred_m1 = m1_model.predict(X_test_m1)

In [69]:
m1_rmse = mean_squared_error(y_test_m1, y_pred_m1, squared=False)
print(f"RMSE: {m1_rmse:.3f}")
print(classification_report(y_test_m1, y_pred_m1))

RMSE: 0.486
              precision    recall  f1-score   support

           0       0.77      0.73      0.75        51
           1       0.76      0.80      0.78        55

    accuracy                           0.76       106
   macro avg       0.76      0.76      0.76       106
weighted avg       0.76      0.76      0.76       106



In [70]:
X_val_m1 = m1_df_validation.drop("Team1_Win", axis=1).values
y_val_m1 = m1_df_validation["Team1_Win"].values

In [71]:
y_pred_val_m1 = m1_model.predict(X_val_m1)

In [72]:
m1_val_rmse = mean_squared_error(y_val_m1, y_pred_val_m1, squared=False)
print(f"RMSE: {m1_val_rmse:.3f}")
print(classification_report(y_val_m1, y_pred_val_m1))

RMSE: 0.514
              precision    recall  f1-score   support

           0       0.78      0.69      0.73       155
           1       0.70      0.78      0.74       144

    accuracy                           0.74       299
   macro avg       0.74      0.74      0.74       299
weighted avg       0.74      0.74      0.74       299



# Model 2
- Makes use of all tourney-level statistics in `tourney_results_detailed`

In [73]:
tourney_results_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12


In [74]:
tourney_results_detailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [75]:
# Use lowest_id to set lowest ID team to Team1
m2_df = tourney_results_detailed.apply(lowest_id, axis=1)
m2_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Team1,Team2
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,3124,3201
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,3173,3395
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,3181,3214
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,3199,3256
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12,3207,3265


In [76]:
# replace W/L columns with appropriate T1 or T2 for each row
def team_cols(row):
  if row['WTeamID'] == row['Team1']:
    row['Team1_Win'] = 1
    for col in m2_df.columns:
      if col != 'WLoc':
        if col[0] == 'W':
          row['T1'+col[1:]] = row[col]
        elif col[0] == 'L':
          row['T2'+col[1:]] = row[col]
  else:
    row['Team1_Win'] = 0
    for col in m2_df.columns:
      if col[0] == 'W':
        row['T2'+col[1:]] = row[col]
      elif col[0] == 'L':
        row['T1'+col[1:]] = row[col]

  return row 

m2_df = m2_df.apply(team_cols, axis=1)
m2_df.drop(['WTeamID', 'WScore', 'LTeamID', 'LScore',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'T1TeamID', 'T2TeamID'], axis=1, inplace=True)
m2_df.head()

,DayNum,Season,T1Ast,T1Blk,T1DR,T1FGA,T1FGA3,T1FGM,T1FGM3,T1FTA,T1FTM,T1OR,T1PF,T1Score,T1Stl,T1TO,T2Ast,T2Blk,T2DR,T2FGA,T2FGA3,T2FGM,T2FGM3,T2FTA,T2FTM,T2Loc,T2OR,T2PF,T2Score,T2Stl,T2TO,Team1,Team1_Win,Team2,WLoc
0,138,2010,22,2,24,57,5,28,1,19,12,13,12,69,6,12,12,1,19,61,34,21,10,5,3,NaN,17,18,55,4,18,3124,1,3201,N
1,138,2010,13,10,34,59,26,23,9,19,12,13,14,67,3,16,8,6,26,73,27,22,8,15,14,NaN,18,22,66,8,8,3173,1,3395,N
2,138,2010,15,7,34,57,13,26,4,22,16,13,11,72,10,11,4,4,21,56,15,15,4,8,3,NaN,10,20,37,6,16,3181,1,3214,H
3,138,2010,13,3,27,63,15,25,3,26,22,20,21,75,8,17,13,4,21,62,20,21,2,22,17,NaN,16,24,61,5,16,3199,1,3256,H
4,138,2010,16,5,29,68,25,24,8,8,6,20,18,62,5,8,9,4,22,60,26,13,5,17,11,NaN,16,12,42,3,10,3207,1,3265,N


In [77]:
m2_df = m2_df.apply(loc_convert, axis=1)
m2_df = m2_df.drop('WLoc', axis=1)
m2_df.head()

,DayNum,Season,T1Ast,T1Blk,T1DR,T1FGA,T1FGA3,T1FGM,T1FGM3,T1FTA,T1FTM,T1OR,T1PF,T1Score,T1Stl,T1TO,T2Ast,T2Blk,T2DR,T2FGA,T2FGA3,T2FGM,T2FGM3,T2FTA,T2FTM,T2Loc,T2OR,T2PF,T2Score,T2Stl,T2TO,Team1,Team1_Win,Team2,Team1_Home,Team1_Neutral
0,138,2010,22,2,24,57,5,28,1,19,12,13,12,69,6,12,12,1,19,61,34,21,10,5,3,NaN,17,18,55,4,18,3124,1,3201,0,1
1,138,2010,13,10,34,59,26,23,9,19,12,13,14,67,3,16,8,6,26,73,27,22,8,15,14,NaN,18,22,66,8,8,3173,1,3395,0,1
2,138,2010,15,7,34,57,13,26,4,22,16,13,11,72,10,11,4,4,21,56,15,15,4,8,3,NaN,10,20,37,6,16,3181,1,3214,1,0
3,138,2010,13,3,27,63,15,25,3,26,22,20,21,75,8,17,13,4,21,62,20,21,2,22,17,NaN,16,24,61,5,16,3199,1,3256,1,0
4,138,2010,16,5,29,68,25,24,8,8,6,20,18,62,5,8,9,4,22,60,26,13,5,17,11,NaN,16,12,42,3,10,3207,1,3265,0,1


In [78]:
m2_df.columns

Index(['DayNum', 'Season', 'T1Ast', 'T1Blk', 'T1DR', 'T1FGA', 'T1FGA3',
       'T1FGM', 'T1FGM3', 'T1FTA', 'T1FTM', 'T1OR', 'T1PF', 'T1Score', 'T1Stl',
       'T1TO', 'T2Ast', 'T2Blk', 'T2DR', 'T2FGA', 'T2FGA3', 'T2FGM', 'T2FGM3',
       'T2FTA', 'T2FTM', 'T2Loc', 'T2OR', 'T2PF', 'T2Score', 'T2Stl', 'T2TO',
       'Team1', 'Team1_Win', 'Team2', 'Team1_Home', 'Team1_Neutral'],
      dtype='object')

Create tournament statistics per season for each team.

In [79]:
m2_game = m2_df[['Team1', 'Team2', 'Season', 'DayNum', 'Team1_Home', 'Team1_Neutral', 'Team1_Win']]
m2_stats = m2_df[['Team1', 'Team2', 'Season','T1Ast', 'T1Blk', 'T1DR', 'T1FGA', 'T1FGA3',
       'T1FGM', 'T1FGM3', 'T1FTA', 'T1FTM', 'T1OR', 'T1PF', 'T1Score',
       'T1Stl', 'T1TO', 'T2Ast', 'T2Blk', 'T2DR', 'T2FGA', 'T2FGA3', 'T2FGM',
       'T2FGM3', 'T2FTA', 'T2FTM', 'T2OR', 'T2PF', 'T2Score', 'T2Stl', 'T2TO']]

In [80]:
m2_stats.head()

,Team1,Team2,Season,T1Ast,T1Blk,T1DR,T1FGA,T1FGA3,T1FGM,T1FGM3,T1FTA,T1FTM,T1OR,T1PF,T1Score,T1Stl,T1TO,T2Ast,T2Blk,T2DR,T2FGA,T2FGA3,T2FGM,T2FGM3,T2FTA,T2FTM,T2OR,T2PF,T2Score,T2Stl,T2TO
0,3124,3201,2010,22,2,24,57,5,28,1,19,12,13,12,69,6,12,12,1,19,61,34,21,10,5,3,17,18,55,4,18
1,3173,3395,2010,13,10,34,59,26,23,9,19,12,13,14,67,3,16,8,6,26,73,27,22,8,15,14,18,22,66,8,8
2,3181,3214,2010,15,7,34,57,13,26,4,22,16,13,11,72,10,11,4,4,21,56,15,15,4,8,3,10,20,37,6,16
3,3199,3256,2010,13,3,27,63,15,25,3,26,22,20,21,75,8,17,13,4,21,62,20,21,2,22,17,16,24,61,5,16
4,3207,3265,2010,16,5,29,68,25,24,8,8,6,20,18,62,5,8,9,4,22,60,26,13,5,17,11,16,12,42,3,10


In [81]:
T1stat_cols = [col for col in m2_stats.columns if col.startswith('T1')]
T2stat_cols = [col for col in m2_stats.columns if col.startswith('T2')]

T1stats = m2_stats[["Season","Team1"]+T1stat_cols]
T2stats = m2_stats[["Season","Team2"]+T2stat_cols]

avg_T1stats = T1stats.groupby(["Season","Team1"])[T1stat_cols].agg(np.mean).reset_index()
avg_T2stats = T2stats.groupby(["Season","Team2"])[T2stat_cols].agg(np.mean).reset_index()
avg_T1stats.head()

,Season,Team1,T1Ast,T1Blk,T1DR,T1FGA,T1FGA3,T1FGM,T1FGM3,T1FTA,T1FTM,T1OR,T1PF,T1Score,T1Stl,T1TO
0,2010,3114,12.0,1.5,22.0,49.0,14.5,19.5,3.0,19.0,11.5,9.0,15.0,53.5,11.0,18.5
1,2010,3122,4.0,3.0,17.0,53.0,14.0,16.0,3.0,9.0,7.0,11.0,13.0,42.0,4.0,20.0
2,2010,3124,14.6,9.0,31.8,50.0,7.0,21.0,1.4,22.2,15.8,8.0,13.8,59.2,5.0,15.6
3,2010,3132,12.0,1.0,25.0,51.0,22.0,19.0,11.0,22.0,13.0,10.0,15.0,62.0,5.0,16.0
4,2010,3151,12.0,1.0,32.0,60.0,34.0,23.0,10.0,11.0,7.0,11.0,17.0,63.0,4.0,23.0


In [82]:
avg_T1stats = avg_T1stats.copy()
for col in avg_T1stats.columns:
  if col.startswith('T1'):
    avg_T1stats.rename(columns={col: col+'_mean'}, inplace=True)

avg_T1stats.head()

,Season,Team1,T1Ast_mean,T1Blk_mean,T1DR_mean,T1FGA_mean,T1FGA3_mean,T1FGM_mean,T1FGM3_mean,T1FTA_mean,T1FTM_mean,T1OR_mean,T1PF_mean,T1Score_mean,T1Stl_mean,T1TO_mean
0,2010,3114,12.0,1.5,22.0,49.0,14.5,19.5,3.0,19.0,11.5,9.0,15.0,53.5,11.0,18.5
1,2010,3122,4.0,3.0,17.0,53.0,14.0,16.0,3.0,9.0,7.0,11.0,13.0,42.0,4.0,20.0
2,2010,3124,14.6,9.0,31.8,50.0,7.0,21.0,1.4,22.2,15.8,8.0,13.8,59.2,5.0,15.6
3,2010,3132,12.0,1.0,25.0,51.0,22.0,19.0,11.0,22.0,13.0,10.0,15.0,62.0,5.0,16.0
4,2010,3151,12.0,1.0,32.0,60.0,34.0,23.0,10.0,11.0,7.0,11.0,17.0,63.0,4.0,23.0


In [83]:
avg_T2stats = avg_T2stats.copy()
for col in avg_T2stats.columns:
  if col.startswith('T2'):
    avg_T2stats.rename(columns={col: col+'_mean'}, inplace=True)

avg_T2stats.head()

,Season,Team2,T2Ast_mean,T2Blk_mean,T2DR_mean,T2FGA_mean,T2FGA3_mean,T2FGM_mean,T2FGM3_mean,T2FTA_mean,T2FTM_mean,T2OR_mean,T2PF_mean,T2Score_mean,T2Stl_mean,T2TO_mean
0,2010,3163,20.0,3.0,25.0,67.0,17.0,28.0,3.0,19.0,11.0,14.0,15.0,70.0,6.0,7.0
1,2010,3181,7.0,4.0,29.0,65.0,17.0,15.0,3.0,20.0,15.0,19.0,16.0,48.0,6.0,11.0
2,2010,3199,10.0,3.0,17.0,63.0,14.0,18.0,4.0,15.0,10.0,14.0,17.0,50.0,2.0,19.0
3,2010,3201,12.0,1.0,19.0,61.0,34.0,21.0,10.0,5.0,3.0,17.0,18.0,55.0,4.0,18.0
4,2010,3207,9.0,5.0,25.0,70.0,18.0,12.0,3.0,10.0,6.0,19.0,24.0,33.0,8.0,13.0


In [84]:
# merge avg stats back with original df 

m2_df = m2_game.merge(avg_T1stats, how='left', on=['Season', 'Team1'])
m2_df = m2_df.merge(avg_T2stats, how='left', on=['Season', 'Team2'])
m2_df.head()

,Team1,Team2,Season,DayNum,Team1_Home,Team1_Neutral,Team1_Win,T1Ast_mean,T1Blk_mean,T1DR_mean,T1FGA_mean,T1FGA3_mean,T1FGM_mean,T1FGM3_mean,T1FTA_mean,T1FTM_mean,T1OR_mean,T1PF_mean,T1Score_mean,T1Stl_mean,T1TO_mean,T2Ast_mean,T2Blk_mean,T2DR_mean,T2FGA_mean,T2FGA3_mean,T2FGM_mean,T2FGM3_mean,T2FTA_mean,T2FTM_mean,T2OR_mean,T2PF_mean,T2Score_mean,T2Stl_mean,T2TO_mean
0,3124,3201,2010,138,0,1,1,14.600000,9.000000,31.800000,50.000000,7.000000,21.000000,1.400000,22.200000,15.8,8.000000,13.800000,59.200000,5.000000,15.600000,12.0,1.0,19.0,61.0,34.0,21.0,10.0,5.0,3.0,17.0,18.0,55.0,4.0,18.0
1,3173,3395,2010,138,0,1,1,11.000000,7.500000,29.500000,62.000000,19.500000,25.000000,6.500000,14.500000,9.0,12.500000,15.000000,65.500000,4.500000,15.500000,8.0,6.0,26.0,73.0,27.0,22.0,8.0,15.0,14.0,18.0,22.0,66.0,8.0,8.0
2,3181,3214,2010,138,1,0,1,10.666667,4.333333,26.000000,56.000000,10.666667,24.333333,2.333333,20.000000,15.0,14.333333,13.666667,66.000000,15.333333,17.666667,4.0,4.0,21.0,56.0,15.0,15.0,4.0,8.0,3.0,10.0,20.0,37.0,6.0,16.0
3,3199,3256,2010,138,1,0,1,16.666667,5.333333,23.333333,59.333333,16.333333,26.333333,5.000000,16.666667,14.0,15.000000,18.666667,71.666667,6.666667,17.333333,13.0,4.0,21.0,62.0,20.0,21.0,2.0,22.0,17.0,16.0,24.0,61.0,5.0,16.0
4,3207,3265,2010,138,0,1,1,16.000000,5.000000,29.000000,68.000000,25.000000,24.000000,8.000000,8.000000,6.0,20.000000,18.000000,62.000000,5.000000,8.000000,9.0,4.0,22.0,60.0,26.0,13.0,5.0,17.0,11.0,16.0,12.0,42.0,3.0,10.0


In [85]:
def add_seeds(row):
  season = row['Season']
  T1 = row['Team1']
  T2 = row['Team2']
  row['T1Seed'] = seeds.query("Season=={0} & TeamID=={1}".format(season, T1))['Seed'].values[0][1:].lstrip('0')
  row['T2Seed'] = seeds.query("Season=={0} & TeamID=={1}".format(season, T2))['Seed'].values[0][1:].lstrip('0')

  return row

m2_df = m2_df.apply(add_seeds, axis=1)
m2_df[['T1Seed', 'T2Seed']] = m2_df[['T1Seed', 'T2Seed']].astype('int')
m2_df['Season'] = m2_df['Season'].astype('int')
m2_df.head()

,Team1,Team2,Season,DayNum,Team1_Home,Team1_Neutral,Team1_Win,T1Ast_mean,T1Blk_mean,T1DR_mean,T1FGA_mean,T1FGA3_mean,T1FGM_mean,T1FGM3_mean,T1FTA_mean,T1FTM_mean,T1OR_mean,T1PF_mean,T1Score_mean,T1Stl_mean,T1TO_mean,T2Ast_mean,T2Blk_mean,T2DR_mean,T2FGA_mean,T2FGA3_mean,T2FGM_mean,T2FGM3_mean,T2FTA_mean,T2FTM_mean,T2OR_mean,T2PF_mean,T2Score_mean,T2Stl_mean,T2TO_mean,T1Seed,T2Seed
0,3124.0,3201.0,2010,138.0,0.0,1.0,1.0,14.600000,9.000000,31.800000,50.000000,7.000000,21.000000,1.400000,22.200000,15.8,8.000000,13.800000,59.200000,5.000000,15.600000,12.0,1.0,19.0,61.0,34.0,21.0,10.0,5.0,3.0,17.0,18.0,55.0,4.0,18.0,4,13
1,3173.0,3395.0,2010,138.0,0.0,1.0,1.0,11.000000,7.500000,29.500000,62.000000,19.500000,25.000000,6.500000,14.500000,9.0,12.500000,15.000000,65.500000,4.500000,15.500000,8.0,6.0,26.0,73.0,27.0,22.0,8.0,15.0,14.0,18.0,22.0,66.0,8.0,8.0,8,9
2,3181.0,3214.0,2010,138.0,1.0,0.0,1.0,10.666667,4.333333,26.000000,56.000000,10.666667,24.333333,2.333333,20.000000,15.0,14.333333,13.666667,66.000000,15.333333,17.666667,4.0,4.0,21.0,56.0,15.0,15.0,4.0,8.0,3.0,10.0,20.0,37.0,6.0,16.0,2,15
3,3199.0,3256.0,2010,138.0,1.0,0.0,1.0,16.666667,5.333333,23.333333,59.333333,16.333333,26.333333,5.000000,16.666667,14.0,15.000000,18.666667,71.666667,6.666667,17.333333,13.0,4.0,21.0,62.0,20.0,21.0,2.0,22.0,17.0,16.0,24.0,61.0,5.0,16.0,3,14
4,3207.0,3265.0,2010,138.0,0.0,1.0,1.0,16.000000,5.000000,29.000000,68.000000,25.000000,24.000000,8.000000,8.000000,6.0,20.000000,18.000000,62.000000,5.000000,8.000000,9.0,4.0,22.0,60.0,26.0,13.0,5.0,17.0,11.0,16.0,12.0,42.0,3.0,10.0,5,12


In [86]:
m2_df.columns

Index(['Team1', 'Team2', 'Season', 'DayNum', 'Team1_Home', 'Team1_Neutral',
       'Team1_Win', 'T1Ast_mean', 'T1Blk_mean', 'T1DR_mean', 'T1FGA_mean',
       'T1FGA3_mean', 'T1FGM_mean', 'T1FGM3_mean', 'T1FTA_mean', 'T1FTM_mean',
       'T1OR_mean', 'T1PF_mean', 'T1Score_mean', 'T1Stl_mean', 'T1TO_mean',
       'T2Ast_mean', 'T2Blk_mean', 'T2DR_mean', 'T2FGA_mean', 'T2FGA3_mean',
       'T2FGM_mean', 'T2FGM3_mean', 'T2FTA_mean', 'T2FTM_mean', 'T2OR_mean',
       'T2PF_mean', 'T2Score_mean', 'T2Stl_mean', 'T2TO_mean', 'T1Seed',
       'T2Seed'],
      dtype='object')

In [87]:
m2_df = m2_df[['Season', 'DayNum', 'Team1_Home', 'Team1_Neutral',
       'Team1_Win', 'T1Ast_mean', 'T1Blk_mean', 'T1DR_mean', 'T1FGA_mean',
       'T1FGA3_mean', 'T1FGM_mean', 'T1FGM3_mean', 'T1FTA_mean', 'T1FTM_mean',
       'T1OR_mean', 'T1PF_mean', 'T1Score_mean', 'T1Stl_mean', 'T1TO_mean',
       'T2Ast_mean', 'T2Blk_mean', 'T2DR_mean', 'T2FGA_mean', 'T2FGA3_mean',
       'T2FGM_mean', 'T2FGM3_mean', 'T2FTA_mean', 'T2FTM_mean', 'T2OR_mean',
       'T2PF_mean', 'T2Score_mean', 'T2Stl_mean', 'T2TO_mean', 'T1Seed',
       'T2Seed']]

In [88]:
m2_df.shape

(693, 35)

In [89]:
m2_df_val = m2_df[m2_df['Season'] >= 2016]
m2_df = m2_df[m2_df['Season'] < 2016]

In [90]:
m2_df.head()

,Season,DayNum,Team1_Home,Team1_Neutral,Team1_Win,T1Ast_mean,T1Blk_mean,T1DR_mean,T1FGA_mean,T1FGA3_mean,T1FGM_mean,T1FGM3_mean,T1FTA_mean,T1FTM_mean,T1OR_mean,T1PF_mean,T1Score_mean,T1Stl_mean,T1TO_mean,T2Ast_mean,T2Blk_mean,T2DR_mean,T2FGA_mean,T2FGA3_mean,T2FGM_mean,T2FGM3_mean,T2FTA_mean,T2FTM_mean,T2OR_mean,T2PF_mean,T2Score_mean,T2Stl_mean,T2TO_mean,T1Seed,T2Seed
0,2010,138.0,0.0,1.0,1.0,14.600000,9.000000,31.800000,50.000000,7.000000,21.000000,1.400000,22.200000,15.8,8.000000,13.800000,59.200000,5.000000,15.600000,12.0,1.0,19.0,61.0,34.0,21.0,10.0,5.0,3.0,17.0,18.0,55.0,4.0,18.0,4,13
1,2010,138.0,0.0,1.0,1.0,11.000000,7.500000,29.500000,62.000000,19.500000,25.000000,6.500000,14.500000,9.0,12.500000,15.000000,65.500000,4.500000,15.500000,8.0,6.0,26.0,73.0,27.0,22.0,8.0,15.0,14.0,18.0,22.0,66.0,8.0,8.0,8,9
2,2010,138.0,1.0,0.0,1.0,10.666667,4.333333,26.000000,56.000000,10.666667,24.333333,2.333333,20.000000,15.0,14.333333,13.666667,66.000000,15.333333,17.666667,4.0,4.0,21.0,56.0,15.0,15.0,4.0,8.0,3.0,10.0,20.0,37.0,6.0,16.0,2,15
3,2010,138.0,1.0,0.0,1.0,16.666667,5.333333,23.333333,59.333333,16.333333,26.333333,5.000000,16.666667,14.0,15.000000,18.666667,71.666667,6.666667,17.333333,13.0,4.0,21.0,62.0,20.0,21.0,2.0,22.0,17.0,16.0,24.0,61.0,5.0,16.0,3,14
4,2010,138.0,0.0,1.0,1.0,16.000000,5.000000,29.000000,68.000000,25.000000,24.000000,8.000000,8.000000,6.0,20.000000,18.000000,62.000000,5.000000,8.000000,9.0,4.0,22.0,60.0,26.0,13.0,5.0,17.0,11.0,16.0,12.0,42.0,3.0,10.0,5,12


In [91]:
X_m2 = m2_df.drop(["Team1_Win"], axis=1).values
y_m2 = m2_df["Team1_Win"].values

In [92]:
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(X_m2, y_m2, test_size=0.3, random_state=42)

In [93]:
m2_model = LogisticRegression(max_iter=500)

In [94]:
m2_model.fit(X_train_m2, y_train_m2)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [95]:
y_pred_m2 = m2_model.predict(X_test_m2)

In [96]:
m2_rmse = mean_squared_error(y_test_m2, y_pred_m2, squared=False)
print(f"RMSE: {m2_rmse:.3f}")
print(classification_report(y_test_m2, y_pred_m2))

RMSE: 0.419
              precision    recall  f1-score   support

         0.0       0.84      0.74      0.79        50
         1.0       0.81      0.89      0.85        64

    accuracy                           0.82       114
   macro avg       0.83      0.82      0.82       114
weighted avg       0.83      0.82      0.82       114



In [97]:
X_val_m2 = m2_df_val.drop("Team1_Win", axis=1).values
y_val_m2 = m2_df_val["Team1_Win"].values

In [98]:
y_pred_val_m2 = m2_model.predict(X_val_m2)

In [99]:
m2_val_rmse = mean_squared_error(y_val_m2, y_pred_val_m2, squared=False)
print(f"RMSE: {m2_val_rmse:.3f}")
print(classification_report(y_val_m2, y_pred_val_m2))

RMSE: 0.402
              precision    recall  f1-score   support

         0.0       0.92      0.74      0.82       158
         1.0       0.78      0.94      0.85       157

    accuracy                           0.84       315
   macro avg       0.85      0.84      0.84       315
weighted avg       0.85      0.84      0.84       315



## Model 2.1 
- Standardize features

In [106]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() # initialize scaler
sc.fit(X_train_m2) # fit on TRAINING DATA ONLY to estimate mean, std

# transform data using estimates from the training data
X_train_m2_1 = sc.transform(X_train_m2)
X_test_m2_1 = sc.transform(X_test_m2)

In [103]:
m2_1_model = LogisticRegression(max_iter=500) # initialize logistic regression model

In [107]:
m2_1_model.fit(X_train_m2_1, y_train_m2) # fit model

LogisticRegression(max_iter=500)

In [109]:
y_pred_m2_1 = m2_1_model.predict(X_test_m2_1) # predict on test set

## Model 2.1 Metrics

In [112]:
m2_1_rmse = mean_squared_error(list(y_test_m2), list(y_pred_m2_1), squared=False)
print(f"RMSE: {m2_1_rmse:.3f}")
print(classification_report(list(y_test_m2), list(y_pred_m2_1)))

RMSE: 0.449
              precision    recall  f1-score   support

         0.0       0.79      0.74      0.76        50
         1.0       0.81      0.84      0.82        64

    accuracy                           0.80       114
   macro avg       0.80      0.79      0.79       114
weighted avg       0.80      0.80      0.80       114



## Model 2.2


*   Feed Forward Neural Network




Create build_model() function

In [119]:
def build_model(n_classes,
                hidden_layer_sizes=[],
                activation='relu',
                optimizer='SGD',
                learning_rate=0.01):
  """Build a multi-class logistic regression model using Keras.

  Args:
    n_classes: Number of output classes in the dataset.
    hidden_layer_sizes: A list with the number of units in each hidden layer.
    activation: The activation function to use for the hidden layers.
    optimizer: The optimizer to use (SGD, Adam).
    learning_rate: The desired learning rate for the optimizer.

  Returns:
    model: A tf.keras model (graph).
  """
  tf.keras.backend.clear_session()
  np.random.seed(0)
  tf.random.set_seed(0)

  # INITIALIZE MODEL
  model = keras.Sequential()
  # INPUT LAYER

  model.add(keras.layers.Dense(
      units = 8,
      input_shape = (X_train_m2_1.shape[1],), #input_shape needs to be a tuple..
      activation = 'relu'
  ))

  # HIDDEN LAYERS
  if len(hidden_layer_sizes) > 0:
    for i in range(len(hidden_layer_sizes)):
      model.add(keras.layers.Dense(
      units=hidden_layer_sizes[i],
      activation=activation 
  ))
      
  # OUTPUT LAYER
  model.add(keras.layers.Dense(
      units=n_classes,
      activation = 'sigmoid'
  ))

  # INITIALIZE OPTIMIZER:

  if optimizer == 'SGD':
    optimizer_param = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  else:
    optimizer_param = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # COMPILE MODEL
  model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer_param, 
                metrics=['accuracy']
                )

  return model

Create model and train on X_NoId_train_std, Y_NoId_train

In [160]:
m2_2_model = build_model(n_classes=2,
                      hidden_layer_sizes=[6,4],
                      activation='relu',
                      optimizer='Adam',
                      learning_rate=0.1)
history = m2_2_model.fit(
    x=X_train_m2_1,
    y=y_train_m2,
    epochs = 10,
    batch_size = 64,
)

Epoch 1/10
5/5 [==============================] - 1s 4ms/step - loss: 0.6847 - accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 0s 4ms/step - loss: 0.5093 - accuracy: 0.7500
Epoch 3/10
5/5 [==============================] - 0s 4ms/step - loss: 0.4080 - accuracy: 0.8220
Epoch 4/10
5/5 [==============================] - 0s 6ms/step - loss: 0.3427 - accuracy: 0.8523
Epoch 5/10
5/5 [==============================] - 0s 7ms/step - loss: 0.2846 - accuracy: 0.8788
Epoch 6/10
5/5 [==============================] - 0s 6ms/step - loss: 0.2677 - accuracy: 0.8939
Epoch 7/10
5/5 [==============================] - 0s 5ms/step - loss: 0.2263 - accuracy: 0.9167
Epoch 8/10
5/5 [==============================] - 0s 4ms/step - loss: 0.2301 - accuracy: 0.9280
Epoch 9/10
5/5 [==============================] - 0s 3ms/step - loss: 0.2890 - accuracy: 0.9015
Epoch 10/10
5/5 [==============================] - 0s 3ms/step - loss: 0.3265 - accuracy: 0.9167


Accuracy after each epoch

In [162]:
print(history.history['accuracy']) 

[0.5, 0.75, 0.8219696879386902, 0.8522727489471436, 0.8787878751754761, 0.8939393758773804, 0.9166666865348816, 0.9280303120613098, 0.9015151262283325, 0.9166666865348816]


Create prediction object for model evaluation


In [163]:
m2_2_pred = m2_2_model.predict(X_test_m2_1)
m2_2_pred = m2_2_pred.argmax(axis=-1)

4/4 [==============================] - 0s 3ms/step


## Model 2.2 Metrics

In [164]:
m2_2_rmse = mean_squared_error(list(y_test_m2), list(m2_2_pred), squared=False)
print(f"RMSE: {m2_2_rmse:.3f}")
print(classification_report(list(y_test_m2), list(m2_2_pred)))

RMSE: 0.408
              precision    recall  f1-score   support

         0.0       0.80      0.82      0.81        50
         1.0       0.86      0.84      0.85        64

    accuracy                           0.83       114
   macro avg       0.83      0.83      0.83       114
weighted avg       0.83      0.83      0.83       114



Look for overfitting:
- Remove 100% colinear columns
- Try a model without any team id
- Look into what team stats actually mean - do any of them reveal scores?

- Add Will's code for every year and merge with detailed
  - Also make an 'intuitive only' model: win/loss ratio? seeds?
- For baseline model - is large # of columns a problem? DOF